# How to develop a package

There are a few ways to create and develop a Julia package. Below is one of them, which I use.

## GitHub

You'd better to have a github account for 

- ease to share your package.
- codes safty. Cloud storage is not easy to lose.
- collaboration with your colleagues.

If you have an account there already , you can just sign in. Or you can sign up GitHub. 

After you logged on GitHub, click new in your homepage. In the next page,
1. choose a reposity name, e.g., `Bin403.jl`
2. add some description
3. select `public`, or `private`. If you want to share your package globally, choose `public`.
4. add a `README` file. This will be the first page of your newly created package. It is markdown.
5. add `.gitignore`. Choose a `.gitignore` of Julia.
6. add a license. I usually use a MIT license.

## Local repo

The local repo can be in `~/.julia/packages`, or in another directory you specified, e.g., `~/Music/packages`. I prefer the later. The reason is that when the packages you are using are messy, you can delete `~/.julia` entirely. It would not be pleasant if you are developing codes there.

In a terminal, or WSL (Windows Subsystem for Linux), goto the directory where you want to host your package to be developed. Thenrun `julia` to enter julia REPL (read-eval-print loop):
```julia
] # to enter the julia package environment
generate BIN403 # the package you are using now
^D # that is, <ctrl> + d to exit julia REPL.
```
Then
```bash
cd BIN403
git init  # probably your first git command, this is to make the current directory as a git repo
git add Project.toml src/BIN403.jl
git commit -am 'init' # your first commit. added two files into the local repo.
git remote add origin git@github.com:xijiang/BIN403.jl # set your remote repo
git config pull.rebase true
git pull origin main # you pull .gitignore, LICENSE, README.md from github to local.
```

## Add a function to `BIN403`
There is already a function called `greet` in `BIN403/src/BIN403.jl`. We will remove it. Instead will add other packages and a recursive kinship calculation function here.

We will modify 3 files here. The first one is `BIN403/Project.toml`. We change the version from `0.1.0` to `0.1.1`. The version number is of pattern `major-sub-minor`. We deem this function as a minor change.

We then create a file `BIN403/src/kinship.jl`. It contents is as below:
```julia
"""
    function kinship(ped, i, j)
---
This function is handy if just to calculate relationship of a few (pairs of) ID.
It can also speed up by adding `Thread.@threads` before your pair loop.
"""
function kinship(ped, i, j)
    (i == 0 || j == 0) && return 0
    ipa, ima = ped[i, :]          # used both for below and the last
    i == j && (return 1 + .5kinship(ped, ipa, ima))
    if i < j
        jpa, jma = ped[j, :]
        return .5(kinship(ped, i, jpa) + kinship(ped, i, jma))
    end
    return .5(kinship(ped, j, ipa) + kinship(ped, j, ima))
end
```

The last file to modify is `BIN403/src/BIN403.jl`. We delete the line for `greet` function, and replace it with 
```julia
include("kinship.jl")
```
Now our package `BIN403` with a function `kinship` is almost ready:

```bash
git commit -am 'kinship' # that what we added
git push origin main # now GitHub has a copy of your code
```
Everybody who has now access to GitHub can now use your `kinship` function of your `BIN403` package.

## How others to use the function
First, they need to install this package. n a Julia REPL:
```julia
] # to enter package environment
add https://github.com/xijiang/BIN403.jl
<backspace> # to return to REPL
using BIN403
```
By the way you use the package `BIN403` above, you have to specify the package name of the function, i.e.:
```julia
BIN403.kinship(ped, 1, 2) # to calculate relationship 1 and 2 in the pedigree ped.
```

To omit `BIN403.` every time you run the `kinship` function, you can either do this by design. That is to `export` this function in your `BIN403` package, or, `import` the function on the client side:

```julia
import BIN403:kinship
```

Suppose we use the latter below. Let use the pedigree used by Henderson in his 1976 paper.

In [2]:
"""
    function kinship(ped, i, j)
---
This function is handy if just to calculate relationship of a few (pairs of) ID.
It can also speed up by adding `Thread.@threads` before your pair loop.
"""
function kinship(ped, i, j)
    (i == 0 || j == 0) && return 0
    ipa, ima = ped[i, :]          # used both for below and the last
    i == j && (return 1 + .5kinship(ped, ipa, ima))
    if i < j
        jpa, jma = ped[j, :]
        return .5(kinship(ped, i, jpa) + kinship(ped, i, jma))
    end
    return .5(kinship(ped, j, ipa) + kinship(ped, j, ima))
end

ped = [0 0
    0 0
    1 0
    1 2
    3 4
    1 4
    5 6]
A = zeros(7, 7)
for i in 1:7
    for j in 1:i
        A[i, j] = A[j, i] = kinship(ped, i, j)
    end
end
@show A

A = [1.0 0.0 0.5 0.5 0.5 0.75 0.625; 0.0 1.0 0.0 0.5 0.25 0.25 0.25; 0.5 0.0 1.0 0.25 0.625 0.375 0.5; 0.5 0.5 0.25 1.0 0.625 0.75 0.6875; 0.5 0.25 0.625 0.625 1.125 0.5625 0.84375; 0.75 0.25 0.375 0.75 0.5625 1.25 0.90625; 0.625 0.25 0.5 0.6875 0.84375 0.90625 1.28125]


7×7 Matrix{Float64}:
 1.0    0.0   0.5    0.5     0.5      0.75     0.625
 0.0    1.0   0.0    0.5     0.25     0.25     0.25
 0.5    0.0   1.0    0.25    0.625    0.375    0.5
 0.5    0.5   0.25   1.0     0.625    0.75     0.6875
 0.5    0.25  0.625  0.625   1.125    0.5625   0.84375
 0.75   0.25  0.375  0.75    0.5625   1.25     0.90625
 0.625  0.25  0.5    0.6875  0.84375  0.90625  1.28125

In the above cell, we actually used a local function `kinship`, as we haven't import the `BIN403` package. It is just for illustration. Package development is somewhat more advanced usage of Julia. It will, however, greatly ease you job later.

As a developer, e.g., accumulate your codes for your own data, you might just run your codes with your local repo. Then you can navigate to your `BIN403` directory:
```julia
] # to enter pkg environment
activate .
<backspace> # to go back to REPL
using Revise # an exerlent pkg I use everyday. It traces your pkg modification and update your pkg env.
using BIN403
# ped = something
BIN403.kinship(ped, 5, 6)
```

I have put 12 notebooks which were on juliabox.com, which is now closed down. To know more julia, I suggest to try those 12 notebooks one by one. Each will take 10-20 minutes. After that, you can start Julia programming with a much better background if you haven't programmed before.